In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [2]:
df = pd.read_csv("../data/adult.csv", na_values="?", keep_default_na=True, index_col=0).dropna()
df = df[
    (
            (df["salary"] == "<=50K") & (df["salary K$"] <= 50) |
            (df["salary"] == ">50K") & (df["salary K$"] > 50) &
            (df["salary K$"] > 0)
    )
]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30162 entries, 0 to 32560
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             30162 non-null  int64 
 1   workclass       30162 non-null  object
 2   education       30162 non-null  object
 3   marital-status  30162 non-null  object
 4   occupation      30162 non-null  object
 5   relationship    30162 non-null  object
 6   race            30162 non-null  object
 7   sex             30162 non-null  object
 8   hours-per-week  30162 non-null  int64 
 9   native-country  30162 non-null  object
 10  salary          30162 non-null  object
 11  salary K$       30162 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 3.0+ MB


,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25


# Task 1
Print the count of men and women in the dataset.

In [3]:
print(f"Men: {df[df['sex'] == 'Male']['sex'].count()}, Women: {df['sex'].loc[lambda sex: sex == 'Female'].count()}")

Men: 20380, Women: 9782


# Task 2
Find the average age of men in dataset

In [4]:
df[df['sex'] == 'Male']['age'].mean()

39.18400392541707

# Task 3
Get the percentage of people from Poland (native-country)

In [5]:
df["native-country"].value_counts(normalize=True)["Poland"]*100

0.18566408063125786

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [11]:
print("mean age salary > 50K:", df[df["salary K$"] > 50]["age"].mean())
print("std age salary > 50K:", df[df["salary K$"] > 50]["age"].std())
print("mean age salary <= 50K:", df[df["salary K$"] <= 50]["age"].mean())
print("std age salary <= 50K:", df[df["salary K$"] <= 50]["age"].std())

mean age salary > 50K: 43.95911028236548
std age salary > 50K: 10.269632835673852
mean age salary <= 50K: 36.60806038668668
std age salary <= 50K: 13.464631257161633


# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [3]:
# write your code for task 5 here

# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [ ]:
# write your code for task 6 here

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [5]:
# write your code for task 7 here

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [6]:
# write your code for task 8 here

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [7]:
# write your code for task 9 here